# Imports

In [20]:
import numpy as np
import scipy.sparse.linalg as sp
import igl
import meshplot
import triangle

# Declare cuves

In [21]:
class Curve:
    # p1 and p2 are tuples representing the start and end points
    # c1 and c2 are tuples representing the colors on each side
    def __init__(self, p1, p2, c1, c2):
        self.p1 = np.array(p1)
        self.p2 = np.array(p2)
        self.c1 = np.array(c1)
        self.c2 = np.array(c2)
    
    # Get n samples points on the curve
    def sample_points(self, n=100):
        points = []
        diff = self.p2 - self.p1
        for i in range(n):
            points.append(self.p1 + diff * i / (n-1))
        return np.array(points)


curves = np.array([
    Curve((0.25, 0.25), (0.25, 0.75), (1.0, 0.0, 0.0), (1.0, 0.0, 0.0)),
    Curve((0.75, 0.25), (0.75, 0.75), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0))
])

In [89]:
# Define boundary
v = np.array(((0.0, 0.0), (1.0, 0.0), (1.0, 1.0), (0.0, 1.0)))
segments = np.array(((0, 1), (1, 2), (2, 3), (3, 0)))
BORDER_COLOR = (0.0, 0.0, 0.0)

constrained_indices = [0, 1, 2, 3]
constrained_red = [BORDER_COLOR[0] for _ in range(4)]
constrained_green = [BORDER_COLOR[1] for _ in range(4)]
constrained_blue = [BORDER_COLOR[2] for _ in range(4)]

# Add curve vertices and segments
for c in curves:
    points = c.sample_points(100)
    for i in range(points.shape[0]):
        constrained_indices.append(v.shape[0] + i)
        constrained_red.append(c.c1[0])
        constrained_green.append(c.c1[1])
        constrained_blue.append(c.c1[2])
    
    curve_segments = []
    for i in range(points.shape[0] - 1):
        curve_segments.append((v.shape[0]+i, v.shape[0]+i+1))
    
    v = np.append(v, points, axis=0)
    segments = np.append(segments, curve_segments, axis=0)

constrained_indices = np.array(constrained_indices)
constrained_red = np.array(constrained_red)
constrained_green = np.array(constrained_green)
constrained_blue = np.array(constrained_blue)

# Create grid
GRID_SIZE = 100
grid = np.zeros(((GRID_SIZE+1)**2, 2))
i = 0
for x in range(GRID_SIZE+1):
    for y in range(GRID_SIZE+1):
        grid[i] = x / GRID_SIZE, y / GRID_SIZE
        i += 1
v = np.append(v, grid, axis=0)

# Triangulate
triangulation = triangle.triangulate(dict(vertices=v[:,:2], segments=segments))

v = np.zeros((v.shape[0], 3))
v[:, :2] = triangulation["vertices"]
f = triangulation["triangles"]

v_b = constrained_indices

## List of all vertex indices
v_all = np.arange(v.shape[0])

## List of interior indices
v_in = np.setdiff1d(v_all, v_b)

## Construct and slice up Laplacian
l = igl.cotmatrix(v, f)
l_ii = l[v_in, :]
l_ii = l_ii[:, v_in]

l_ib = l[v_in, :]
l_ib = l_ib[:, v_b]

## Dirichlet boundary conditions from z-coordinate
bc = constrained_red
print(v)

## Solve PDE
z_in = sp.spsolve(-l_ii, l_ib.dot(bc))

"""
v_all = np.arange(v.shape[0])
v_nc = np.setdiff1d(v_all, constrained_indices)

l = igl.cotmatrix(v, f)
l_ii = l[v_nc, :]
l_ii = l_ii[:, v_nc]
l_ib = l[v_nc, :]
l_ib = l_ib[:, constrained_indices]

b = l_ib.dot(constrained_red)
z_in = sp.spsolve(-l_ii, b)
"""

meshplot.plot(v, f, z, shading={"wireframe": True})
#p.add_points(v, shading={"point_color" : "green", "point_size": 0.05})

[[0.   0.   0.  ]
 [1.   0.   0.  ]
 [1.   1.   0.  ]
 ...
 [1.   0.98 0.  ]
 [1.   0.99 0.  ]
 [1.   1.   0.  ]]
[nan nan nan ... nan nan nan]
Invalid color array given! Supported are numpy arrays. <class 'numpy.ndarray'>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…